### Data mining

In [1]:
import pandas as pd
import thinkplot
import numpy as np
import statsmodels.formula.api as smf

Read in data

In [2]:
df_raw = pd.read_csv('../../Data/usa_00002.csv')

In [3]:
df_raw.head()

,YEAR,DATANUM,SERIAL,CBSERIAL,HHWT,REGION,STATEICP,STATEFIP,COUNTYICP,COUNTYFIP,...,DIFFHEAR,PWSTATE2,PWCOUNTY,PWMET13,PWTYPE,PWPUMA00,TRANWORK,TRANTIME,DEPARTS,ARRIVES
0,2017,1,1,2017000000016,206,32,41,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1,2017,1,2,2017000000031,45,32,41,1,0,0,...,1,1,117,13820,5,1200,10,50,642,734
2,2017,1,3,2017000000061,136,32,41,1,0,0,...,1,13,0,0,5,1700,10,45,1805,1849
3,2017,1,3,2017000000061,136,32,41,1,0,0,...,1,0,0,0,0,0,0,0,0,0
4,2017,1,3,2017000000061,136,32,41,1,0,0,...,1,0,0,0,0,0,0,0,0,0


In [4]:
work_force = df_raw[df_raw.EMPSTAT == 1] # Consider all employed people

Remove NaNs

In [5]:
work_force = work_force.dropna()
work_force.head()

,YEAR,DATANUM,SERIAL,CBSERIAL,HHWT,REGION,STATEICP,STATEFIP,COUNTYICP,COUNTYFIP,...,DIFFHEAR,PWSTATE2,PWCOUNTY,PWMET13,PWTYPE,PWPUMA00,TRANWORK,TRANTIME,DEPARTS,ARRIVES
1,2017,1,2,2017000000031,45,32,41,1,0,0,...,1,1,117,13820,5,1200,10,50,642,734
2,2017,1,3,2017000000061,136,32,41,1,0,0,...,1,13,0,0,5,1700,10,45,1805,1849
5,2017,1,4,2017000000158,19,32,41,1,0,0,...,1,1,0,0,9,2500,10,25,717,744
10,2017,1,5,2017000000159,21,32,41,1,0,0,...,1,1,0,0,9,290,10,60,602,704
11,2017,1,5,2017000000159,21,32,41,1,0,0,...,1,1,0,0,9,290,10,85,702,829


### Predictive powers

In [6]:
t = []
for name in work_force.columns:
    try:
        if work_force[name].var() < 1e-7:
            continue
        
        formula = 'INCWAGE ~ ' + name
        model = smf.ols(formula, data=work_force)
        if model.nobs < len(work_force)/2:
            continue
            
        results = model.fit()
    except (ValueError, TypeError):
        continue
        
    t.append((results.rsquared, name))

In [7]:
t.sort(reverse=True)
for mse, name in t[:-1]:
    print(name, mse)

INCWAGE 1.0
INCEARN 0.893082765421639
INCTOT 0.8287993208838135
OCCSCORE 0.16741703416287168
SEI 0.11296051232322724
UHRSWORK 0.11199995857540535
EDUCD 0.11048437651301812
EDUC 0.1061655886594568
DEGFIELDD 0.08860226672501026
DEGFIELD 0.08855555915460855
OCC1950 0.06414893004732869
OCC2010 0.06372903051841838
OCC 0.0636359416576765
WKSWORK2 0.0532750175079012
MARST 0.04386789805130986
YRMARR 0.03649606243173387
BIRTHYR 0.025007780903928523
AGE 0.025007780903928523
RELATE 0.02327870652017694
RELATED 0.023180286995274968
SEX 0.02257473842728086
PERNUM 0.022048491705418782
SCHOOL 0.020978971589079354
GRADEATTD 0.019105692154608
GRADEATT 0.018800505725589556
SCHLTYPE 0.017102367913148342
FERTYR 0.014770756185099265
ELDCH 0.012902016601617938
YNGCH 0.012567804309799868
TRANTIME 0.010623289160095473
NCHILD 0.008986469177179046
DEGFIELD2D 0.008543979192768503
DEGFIELD2 0.00854383507123635
PWTYPE 0.008036622759191259
MET2013 0.007912271440267737
PWMET13 0.0076320801358478185
GQ 0.0071773023710

### Blinder–Oaxaca decomposition

In [19]:
temp = work_force.copy()
temp['INCWAGE'].replace([0, 999999, 999998], np.nan, inplace=True)
temp['INCWAGE'] = np.log(temp['INCWAGE'])
temp = temp.dropna()
dependent = 'INCWAGE'

In [20]:
temp['SEX'].describe()

count    1.405825e+06
mean     1.483197e+00
std      4.997177e-01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      2.000000e+00
max      2.000000e+00
Name: SEX, dtype: float64

In [21]:
## Quite confused by this!!!

# Variables to drop
# work_force_clean = temp.copy().drop(columns=['YEAR', 'DATANUM', 'SERIAL', 'CBSERIAL', 'INCEARN', 'INCTOT', 'OCCSCORE', 
#                                              'SEI', 'EDUC', 'DEGFIELD', 'OCC1950', 'OCC', 'GRADEATT', 'DEGFIELD2', 
#                                              'HISPAND', 'RACED', 'WKSWORK2', 'UHRSWORK'])

In [22]:
# Variables to keep
work_force_clean = temp[[dependent, 'EDUCD', 'DEGFIELDD', 'OCC2010', 'MARST', 'AGE', 'SEX', 'WKSWORK2', 'UHRSWORK',
                        'METRO', 'RACE', 'HISPAN', 'IND1990', 'PWTYPE']]

In [38]:
def run_regression(df, dependent):
    formula = dependent + ' ~ '
    for i in range(len(df.columns)):
        name = df.columns[i]
        try:
            if df[name].var() < 1e-7 or name == dependent or name == 'SEX':
                continue

            formula += name
            if i != len(df.columns) - 1:
                formula += ' + '

        except (ValueError, TypeError):
            continue

    model = smf.ols(formula, data=df)
    results = model.fit()
    return results

In [39]:
work_force_male = work_force_clean[work_force_clean['SEX'] == 1]
work_force_female = work_force_clean[work_force_clean['SEX'] == 2]

In [40]:
results_male = run_regression(work_force_male, dependent)
results_female = run_regression(work_force_female, dependent)

In [41]:
total_log_pts = work_force_male[dependent].mean() - work_force_female[dependent].mean()
total_log_pts

0.3875618852347138

In [42]:
explained = 0
for key, val in results_male.params.iteritems():
    if key == 'Intercept':
        continue

    percent = val*(work_force_male[key].mean() - work_force_female[key].mean())/total_log_pts*100
    explained += percent
    print(key, percent)

explained

EDUCD -7.736066355562847
DEGFIELDD -0.883001490895001
OCC2010 -8.65415011926756
MARST 3.4807210959947015
AGE 0.385006439974509
WKSWORK2 13.146855500158486
UHRSWORK 38.8895565656722
METRO 0.04241135521710955
RACE 0.00014968356658429067
HISPAN -0.0532287915999084
IND1990 10.217957742897516
PWTYPE -0.1505892028225454


48.68562242333325

In [50]:
# Unexplained factors
unexplained = 0
for key, val in results_female.params.iteritems():
    if key == 'Intercept':
        continue
    print(work_force_female[key].mean()*(results_male.params[key] - results_female.params[key]))
    unexplained += work_force_female[key].mean()*(results_male.params[key] - results_female.params[key])

unexplained = unexplained/total_log_pts*100
unexplained

0.04615535256600591
-0.01097044299163115
0.04567336044572637
-0.09235833827080386
-0.010771654122708575
0.155293574508504
-0.3103805218937031
-7.978208509149528e-05
-0.01117786011448956
-0.00103162275400146
-0.05134563552663475
-0.0058874762988043354


-63.70106451208864

In [49]:
explained + unexplained

-15.015442088755393

This is bogus: https://github.com/chrisdfong/Gender-Pay-Gap-Analysis/blob/master/Project.ipynb